In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

from tensorflow import keras
from PIL import Image

In [2]:
data = pd.read_csv("river_data.csv")

data['SensorTime']= pd.to_datetime(data['SensorTime'])
data.head()

,SensorTime,CaptureTime,Filename,Agency,SiteNumber,TimeZone,Stage,Discharge,CalcTimestamp,width,...,WeirPt2X,WeirPt2Y,WwRawLineMin,WwRawLineMax,WwRawLineMean,WwRawLineSigma,WwCurveLineMin,WwCurveLineMax,WwCurveLineMean,WwCurveLineSigma
0,2012-06-09 13:15:00,2012-06-09T13:09:07,StateLineWeir_20120609_Farrell_001.jpg,USGS,6674500,MDT,2.99,916.0,2020-03-11T16:58:28,4288,...,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2012-06-09 13:15:00,2012-06-09T13:10:29,StateLineWeir_20120609_Farrell_002.jpg,USGS,6674500,MDT,2.99,916.0,2020-03-11T16:58:33,4288,...,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2012-06-09 13:45:00,2012-06-09T13:44:01,StateLineWeir_20120609_Farrell_003.jpg,USGS,6674500,MDT,2.96,873.0,2020-03-11T16:58:40,4288,...,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2012-06-09 14:45:00,2012-06-09T14:44:30,StateLineWeir_20120609_Farrell_004.jpg,USGS,6674500,MDT,2.94,846.0,2020-03-11T16:58:47,4288,...,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2012-06-09 15:45:00,2012-06-09T15:44:59,StateLineWeir_20120609_Farrell_005.jpg,USGS,6674500,MDT,2.94,846.0,2020-03-11T16:58:55,4288,...,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
train = pd.read_csv("./data/train_data.csv")
test = pd.read_csv("./data/test_data.csv")
validation = pd.read_csv("./data/validation_data.csv")

In [4]:
labels_train = train[[" Filename", " Stage"]]
labels_test = test[[" Filename", " Stage"]]
labels_validation = validation[[" Filename", " Stage"]]
generator = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

train_generator=generator.flow_from_dataframe(
    dataframe=labels_train,
    directory="./data/images/train",
    x_col=" Filename",
    y_col=" Stage",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(512,512))

test_generator = generator.flow_from_dataframe(
    dataframe=labels_test,
    directory="./data/images/test",
    x_col=" Filename",
    y_col=" Stage",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(512,512))

validation_generator = generator.flow_from_dataframe(
    dataframe=labels_validation,
    directory="./data/images/validation",
    x_col=" Filename",
    y_col=" Stage",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(512,512))


Found 25235 validated image filenames.
Found 12618 validated image filenames.
Found 4206 validated image filenames.


In [5]:
im = Image.open("./images/train/StateLineWeir_20120609_Farrell_001.jpg")
im2 = im.crop((80,100,400,450))
im2.show()

In [6]:
inputs = keras.Input(shape=(512, 512, 3))
x = keras.layers.Cropping2D(cropping=((80,112),(100,62)))(inputs)
x = keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(x)
x = keras.layers.MaxPool2D()(x)
x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = keras.layers.MaxPool2D()(x)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dense(64, activation='relu')(x)
outputs = keras.layers.Dense(1, activation='linear')(x)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-11-17 15:42:19.980651: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-17 15:42:19.981097: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss='mse', 
    optimizer='adam', 
    metrics=['mse', 'mae']
)
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    batch_size=32,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/10


2022-11-17 15:42:22.302121: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-17 15:42:22.605319: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


789/789 [==============================] - ETA: 0s - loss: 0.6906 - mse: 0.6906 - mae: 0.5777

2022-11-17 15:43:40.798534: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


789/789 [==============================] - 91s 113ms/step - loss: 0.6906 - mse: 0.6906 - mae: 0.5777 - val_loss: 0.4206 - val_mse: 0.4206 - val_mae: 0.4655
Epoch 2/10
789/789 [==============================] - 89s 112ms/step - loss: 0.3825 - mse: 0.3825 - mae: 0.3905 - val_loss: 0.3504 - val_mse: 0.3504 - val_mae: 0.4120
Epoch 3/10
789/789 [==============================] - 86s 109ms/step - loss: 0.3336 - mse: 0.3336 - mae: 0.3620 - val_loss: 0.3074 - val_mse: 0.3074 - val_mae: 0.3372
Epoch 4/10
789/789 [==============================] - 86s 109ms/step - loss: 0.3110 - mse: 0.3110 - mae: 0.3478 - val_loss: 0.2985 - val_mse: 0.2985 - val_mae: 0.3594
Epoch 5/10
789/789 [==============================] - 85s 108ms/step - loss: 0.3024 - mse: 0.3024 - mae: 0.3459 - val_loss: 0.2921 - val_mse: 0.2921 - val_mae: 0.3149
Epoch 6/10
789/789 [==============================] - 86s 109ms/step - loss: 0.2870 - mse: 0.2870 - mae: 0.3337 - val_loss: 0.2872 - val_mse: 0.2872 - val_mae: 0.3591
Epoch 7/1

In [8]:
rmse = np.sqrt(model.evaluate(test_generator, verbose=0))
print("Test RMSE: ",rmse)

Test RMSE:  [0.3988882  0.3988882  0.50323954]
